# Dividir e conquistar
Às vezes, os programas se tornam muito longos e difíceis de ler. Falamos de [código espaguete](https://en.wikipedia.org/wiki/Spaghetti_code). Uma maneira de tornar o código mais fácil de ler e manter é dividi-lo em funções menores e usá-las em fluxos de trabalho mais complexos. O princípio de design de software é chamado de [Dividir e conquistar](https://www.quora.com/What-is-divide-and-conquer-programming-strategy).

In [1]:
from skimage.io import imread
from skimage.morphology import white_tophat, disk
from skimage.filters import gaussian, threshold_otsu
from skimage.measure import label, regionprops_table
import pandas as pd
import numpy as np

In [2]:
image = imread("../../data/blobs.tif")
footprint = disk(15)
background_subtracted = white_tophat(image, 
                                     footprint=footprint)
particle_radius = 5
denoised = gaussian(background_subtracted, 
                    sigma=particle_radius)
binary = denoised > threshold_otsu(denoised)
labels = label(binary)
requested_measurements = ["label", "area", "mean_intensity"]
regionprops = regionprops_table(image, 
                                labels, 
                                properties=requested_measurements)
table = pd.DataFrame(regionprops)
mean_total_intensity = np.mean(table["area"] * table["mean_intensity"])
mean_total_intensity

17136.90322580645

Uma maneira comum e fácil de tornar esse código mais legível é dividi-lo em seções que começam com um comentário cada.

In [3]:
# configuration
file_to_analyze = "../../data/blobs.tif"
background_subtraction_radius = 15
particle_radius = 5
requested_measurements = ["area", "mean_intensity"]

# load data
image = imread(file_to_analyze)

# preprocess image
footprint = disk(background_subtraction_radius)
background_subtracted = white_tophat(image, 
                                     footprint=footprint)
denoised = gaussian(background_subtracted, 
                    sigma=particle_radius)

# segment image
binary = denoised > threshold_otsu(denoised)
labels = label(binary)

# extract features
regionprops = regionprops_table(image, 
                                labels, 
                                properties=requested_measurements)
table = pd.DataFrame(regionprops)

# descriptive statistics
mean_total_intensity = np.mean(table["area"] * table["mean_intensity"])
mean_total_intensity

17136.90322580645

Mais profissional seria colocar todo esse código em sub-rotinas significativas e chamá-las de uma função central.

In [4]:
# reusable functions
def preprocess_image(image, background_subtraction_radius, particle_radius):
    """Aplicar remoção de fundo e redução de ruído"""
    footprint = disk(background_subtraction_radius)
    background_subtracted = white_tophat(image, footprint=footprint)
    denoised = gaussian(background_subtracted, sigma=particle_radius)
    return denoised

def segment_image(image):
    """Aplicar limiarização e análise de componentes conectados"""
    binary = image > threshold_otsu(image)
    labels = label(binary)
    return labels

def extract_features(image, labels, requested_measurements):
    """Medir propriedades especificadas"""
    regionprops = regionprops_table(image, 
                                    labels, 
                                    properties=requested_measurements)
    table = pd.DataFrame(regionprops)
    return table

Depois de colocarmos grupos de etapas de processamento em funções, podemos chamá-las de uma função principal. Essa função pode ser reutilizada posteriormente para aplicar o mesmo procedimento a todas as imagens em uma pasta. Ela também serve como índice, uma visão geral do fluxo de trabalho de processamento de imagens. Ao ler apenas essa função, sabemos todas as etapas de processamento e quais parâmetros elas têm.

In [5]:
def analyse_average_total_intensity(filename, 
                                    background_subtraction_radius = 15, 
                                    particle_radius = 5):
    """Carregar uma imagem, segmentar objetos e medir sua intensidade total média."""
    image = imread(filename)
    denoised = preprocess_image(image, 
                                background_subtraction_radius, 
                                particle_radius)
    labels = segment_image(denoised)
    requested_measurements = ["area", "mean_intensity"]
    table = extract_features(image, 
                             labels, 
                             requested_measurements)

    # descriptive statistics
    mean_total_intensity = np.mean(table["area"] * table["mean_intensity"])
    
    return mean_total_intensity

In [6]:
# configuration
file_to_analyze = "../../data/blobs.tif"

Essa função central pode então ser lida facilmente; ela tem apenas 6 linhas de código

In [7]:
analyse_average_total_intensity(file_to_analyze)

17136.90322580645

Essa função pode então ser reutilizada também para outros arquivos de imagem.

In [8]:
analyse_average_total_intensity("../../data/BBBC007_batch/20P1_POS0005_D_1UL.tif")

884.2620087336245